In [1]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import math

BATCH_SIZE = 64
MODEL_SAVE_PATH = "cnn_model_params_casia.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [2]:
!rm -rf CASIA CASIA_faceAntisp.rar
!ls

sample_data


In [3]:
!wget https://www.dropbox.com/s/aaz282d9wyst0w8/CASIA_faceAntisp.rar

--2022-11-27 08:48:49--  https://www.dropbox.com/s/aaz282d9wyst0w8/CASIA_faceAntisp.rar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/aaz282d9wyst0w8/CASIA_faceAntisp.rar [following]
--2022-11-27 08:48:49--  https://www.dropbox.com/s/raw/aaz282d9wyst0w8/CASIA_faceAntisp.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb156696aef706b46d9c6bc676a.dl.dropboxusercontent.com/cd/0/inline/BxjcuLpM9PRJSYFWdF6gBHkGNygSBmjmLEVL23tQToVfa3EuP3wG8b3TJZv_3ifZsThKXcORIbcmf_kO2HQR8uEjTvTr0s_U0aFkTXUYeN57B_dliH-KIneK1KaXauEUr29yoqVPzpCFCKZa95VI8Fzz_B0yO7VDB0CA7gQR6wfNeg/file# [following]
--2022-11-27 08:48:50--  https://ucb156696aef706b46d9c6bc676a.dl.dropboxusercontent.com/cd/0/inline/BxjcuLpM9PRJSYFWdF6gBHkGNygSBmjmLEVL23tQToVfa3EuP3wG8

In [4]:
!unrar x CASIA_faceAntisp.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from CASIA_faceAntisp.rar

You can download the datset from here : https://pan.baidu.com/s/15HyX7tizCCuwN9BKiV9_zA
Password: h5un
however, the labels are missing!

see https://github.com/mnikitin/Learn-Convolutional-Neural-Network-for-Face-Anti-Spoofing/issues/5#issuecomment-667916800

Creating    test_release                                              OK
Creating    test_release/18                                           OK
Extracting  test_release/18/1.avi                                          0%  OK 
Creating    train_release                                             OK
Creating    train_release/19                                          OK
Extracting  train_release/19/1.avi                                         0%  OK 
Creating    train_release/17                                          OK
Extracting  train_release/17/1.avi                                         0%

In [5]:
!mkdir CASIA
!mv train_release CASIA && mv test_release CASIA

In [6]:
DATA_DIR = "CASIA/"
IMG_SIZE = 256
NUM_FRAMES_PER_VIDEO = 16
targets = {"1": 0, "2": 0, "3": 1, "4": 1, "5": 2, "6": 2, "7": 3, "8": 3, "HR_1": 0, "HR_2": 1, "HR_3": 2, "HR_4": 3}

def video2frames(video_path, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    is_there_frame = True
    num_total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)    
    resampling_rate = int(num_total_frames / NUM_FRAMES_PER_VIDEO)
    idf = 0
    while is_there_frame and len(frames) < NUM_FRAMES_PER_VIDEO:
        is_there_frame, frame = cap.read()
        if is_there_frame and idf % resampling_rate == 0:
            # grayscale
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.resize(frame, resize)
            # resize
            frames.append(frame)
        idf = idf + 1

    assert len(frames)==NUM_FRAMES_PER_VIDEO
    return np.array(frames)

def load_casia_dataset():
    try:
        num_folders = 1
        dataset_x = []
        dataset_y = []
        files = glob.glob(f'{DATA_DIR}/**/*.avi', recursive=True)
        for f in files:
            frames = video2frames(f)
            dataset_x = [*dataset_x, *frames]
            dataset_x = np.array(dataset_x)
            vector_path = f.split("/")
            tmp = [targets[vector_path[-1][:-4]]]*NUM_FRAMES_PER_VIDEO
            dataset_y = [*dataset_y, *tmp]
    except LookupError: # IndexError, KeyError
        print("index from video name not found in map targets")
    return dataset_x, dataset_y

In [7]:
%%time
dataset_x, dataset_y = load_casia_dataset()

CPU times: user 4min 7s, sys: 17.4 s, total: 4min 24s
Wall time: 2min 59s


In [8]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, tensors, transform=None):
      assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
      self.tensors = tensors
      self.transform = transform

  def __getitem__(self, index):
      x = self.tensors[0][index]

      if self.transform:
          x = self.transform(x)

      y = self.tensors[1][index]

      return x, y

  def __len__(self):
      return self.tensors[0].size(0)

In [9]:
%%time
transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])
combined_set = CustomDataset((torch.Tensor(dataset_x), torch.as_tensor(dataset_y)), transform=transform)
train_set, test_set = torch.utils.data.random_split(combined_set, [7680, 1920])

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

CPU times: user 414 ms, sys: 510 ms, total: 924 ms
Wall time: 923 ms


In [10]:
img, label = train_set[0]
img, label

(tensor([[[0.4353, 0.4353, 0.4353,  ..., 0.4039, 0.4039, 0.4039],
          [0.4353, 0.4353, 0.4353,  ..., 0.4039, 0.4039, 0.4039],
          [0.4353, 0.4353, 0.4353,  ..., 0.4039, 0.4039, 0.4039],
          ...,
          [0.7569, 0.7647, 0.7608,  ..., 0.7412, 0.7412, 0.7412],
          [0.7647, 0.7686, 0.7686,  ..., 0.7216, 0.7216, 0.7216],
          [0.8235, 0.8275, 0.8275,  ..., 0.7176, 0.7176, 0.7176]]]), tensor(0))

In [11]:
num_classes = 4
learning_rate = 0.001
num_epochs = 20

class CNN(nn.Module):
  def __init__(self, num_classes=0):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(64*64*32, num_classes)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

In [12]:
def train(model, optimizer, loss_fn, num_epochs):
  loss_vals = []
  running_loss = 0.0
  total_step = len(train_loader)

  list_loss = []
  list_time = []
  j = 0

  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)

      # forward
      output = model(images)
      loss = loss_fn(output, labels)

      # change the params
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      list_loss.append(loss.item())
      list_time.append(j)
      j += 1

      if (i + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{ num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():4f}")

  print("Finished Training Trainset")
  return list_loss

In [13]:
model = CNN(num_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

print([e.shape for e in model.fc.parameters()])
model.fc.weight
list_loss = train(model, optimizer, loss_fn, 20)

[torch.Size([4, 131072]), torch.Size([4])]
Epoch [1/20], Step [100/120], Loss: 0.273247
Epoch [2/20], Step [100/120], Loss: 0.148622
Epoch [3/20], Step [100/120], Loss: 0.027702
Epoch [4/20], Step [100/120], Loss: 0.105921
Epoch [5/20], Step [100/120], Loss: 0.017623
Epoch [6/20], Step [100/120], Loss: 0.074360
Epoch [7/20], Step [100/120], Loss: 0.035689
Epoch [8/20], Step [100/120], Loss: 0.019176
Epoch [9/20], Step [100/120], Loss: 0.006849
Epoch [10/20], Step [100/120], Loss: 0.040258
Epoch [11/20], Step [100/120], Loss: 0.029746
Epoch [12/20], Step [100/120], Loss: 0.019096
Epoch [13/20], Step [100/120], Loss: 0.021618
Epoch [14/20], Step [100/120], Loss: 0.046113
Epoch [15/20], Step [100/120], Loss: 0.000301
Epoch [16/20], Step [100/120], Loss: 0.017398
Epoch [17/20], Step [100/120], Loss: 0.050566
Epoch [18/20], Step [100/120], Loss: 0.000666
Epoch [19/20], Step [100/120], Loss: 0.000057
Epoch [20/20], Step [100/120], Loss: 0.043077
Finished Training Trainset


In [14]:
with torch.no_grad():
  correct =0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print(f"Test Accuracy of the model on the 1920 test images: {100 * correct / total}")

Test Accuracy of the model on the 1920 test images: 98.80208333333333


In [16]:
torch.save(model.state_dict(), MODEL_SAVE_PATH)